In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import pprint
from __future__ import division
from __future__ import print_function
import calendar
from zipfile import ZipFile
import os
import sys
import dateutil
import zipfile
import csv
import pickle
import calendar
from datetime import datetime

In [2]:
def collect_routerData(routerDataFile , base_data=pd.DataFrame()):
    filePath = routerDataFile
    directionIsIn = False
    data = base_data.copy(deep=True)
    group = ""
    routerNum = 0
    readebleFormat = True
    with open(filePath, "rb") as csvfile:
        csvReader = csv.reader(csvfile, delimiter=',')
        for row in csvReader:
            #print(readebleFormat)
            #print(group_and_router + str(directionIsIn))
            # check if the line defines our router
            if len(row) == 1 and "cern.ch" in row[0]:
                routerNum += 1
                continue
            # check if there is actually data
            if len(row) != 2:
                continue
            # check if the line defines our user group
            if row[0] == "Subtitle:":
                group = row[1].split("User Group: ")[1]
            # check if we can parse this data
            if row[0] == "View Title:" and row[1] == "In and Out traffic for each link":
                readebleFormat = True
            elif row[0] == "View Title:":
                readebleFormat = False
            #if row[0] == "View Title:" and row[1] == "Total In traffic - Interface - Bits In - Average Rate - Descending":
            #    readebleFormat = False
            #if row[0] == "View Title:" and row[1] == "Total Out traffic - Interface - Bits Out - Average Rate - Descending":
            #    readebleFormat = False
            if readebleFormat == False:
                continue
            # check the direction
            if row[1] == "Bits In - Average Rate (Bits Per Second)":
                directionIsIn = True
            if row[1] == "Bits Out - Average Rate (Bits Per Second)":
                directionIsIn = False
            try:
                float(row[1])
            except ValueError:
                continue
            bitsPerSec = float(row[1])
            row[0] = row[0].replace(" ", "/")
            # the format for the years is changed inbetween from 16 to 2016
            year = row[0].split("/")[2]
            if len(year) == 4:
                timestomp = datetime.strptime(row[0], "%m/%d/%Y/%H:%M")
            else:
                timestomp = datetime.strptime(row[0], "%m/%d/%y/%H:%M")
            # add data to dataFrame
            # correcting timestamp for cest to gmt
            timestamp_epoch = int(calendar.timegm(timestomp.utctimetuple()))-60*60*2
            if directionIsIn:
                data.set_value(timestamp_epoch, "throughput_"+group+"_router_"+str(routerNum)+"_in", bitsPerSec)
            else:
                data.set_value(timestamp_epoch, "throughput_"+group+"_router_"+str(routerNum)+"_out", bitsPerSec)
    data.sort_index(inplace=True, ascending=False)
    return data


In [3]:
def readAllInFolder(folderPath, base_data=pd.DataFrame()):
    data = base_data
    for (dirpath, dirnames, filenames) in os.walk(folderPath):
        # read all files
        for filename in filenames:
            data = collect_routerData(folderPath + filename, base_data=data)
    return data

In [4]:
data_router = readAllInFolder("raw_data_routers/PIC/")
data_router = readAllInFolder("raw_data_routers/RAL/", base_data=data_router)
data_router = readAllInFolder("raw_data_routers/TRIUMF/", base_data=data_router)
data_router = readAllInFolder("raw_data_routers/KIT/", base_data=data_router)
data_router = readAllInFolder("raw_data_routers/CCIN2P3/", base_data=data_router)
data_router = readAllInFolder("raw_data_routers/BNL/", base_data=data_router)
data_router = readAllInFolder("raw_data_routers/CNAF/", base_data=data_router)
data_router = readAllInFolder("raw_data_routers/JINR-T1/", base_data=data_router)

data_router.to_pickle("raw_router_CERN_to_RAL-PIC-TRIUMF-KIT-IN2P3-BNL-CNAF-JINR-T1_highres.pkl")
data_router

,throughput_ES-PIC_router_1_in,throughput_ES-PIC_router_1_out,throughput_UK-RAL_router_1_in,throughput_UK-RAL_router_1_out,throughput_UK-RAL_router_2_in,throughput_UK-RAL_router_2_out,throughput_CA-TRIUMF_router_1_in,throughput_CA-TRIUMF_router_1_out,throughput_DE-KIT_router_1_in,throughput_DE-KIT_router_1_out,...,throughput_IT-INFN-CNAF_router_2_in,throughput_IT-INFN-CNAF_router_2_out,throughput_IT-INFN-CNAF_router_3_in,throughput_IT-INFN-CNAF_router_3_out,throughput_IT-INFN-CNAF_router_4_in,throughput_IT-INFN-CNAF_router_4_out,throughput_JINR-T1_router_1_in,throughput_JINR-T1_router_1_out,throughput_JINR-T1_router_2_in,throughput_JINR-T1_router_2_out
1475445600,1.121984e+09,1.496550e+09,5.451270e+09,1.495857e+09,1.754114e+09,1.351659e+09,5.604321e+08,2.565073e+08,6.945322e+08,2.618570e+09,...,5.037367e+08,4.070514e+09,2.482879e+08,4.297731e+09,2.640128e+08,4.504391e+09,4.901760e+09,4.870247e+09,88.693333,88.693333
1475445300,1.087122e+09,2.998886e+09,4.437934e+09,6.365802e+09,1.472997e+09,4.201969e+09,9.558961e+08,2.009845e+09,2.792970e+09,2.381259e+09,...,9.100900e+08,5.190873e+09,6.477239e+08,4.692290e+09,1.917753e+08,5.056410e+09,5.159029e+09,5.135390e+09,83.653333,83.653333
1475445000,1.651072e+09,5.505669e+09,3.023149e+09,5.210100e+09,3.700444e+08,4.315129e+09,2.839772e+08,1.177839e+08,4.833703e+09,1.765045e+09,...,4.556132e+08,3.755450e+09,1.031895e+09,3.927576e+09,1.245936e+09,5.189995e+09,3.751000e+09,3.718323e+09,120.000000,120.000000
1475444700,1.066699e+09,7.867032e+09,1.401690e+09,5.534843e+09,5.011927e+08,4.505527e+09,4.171580e+08,2.186001e+09,4.152914e+09,2.206540e+09,...,1.247090e+08,3.896233e+09,1.435699e+08,3.937131e+09,2.095743e+08,4.341368e+09,4.255231e+09,4.229704e+09,88.693333,88.693333
1475444400,2.213880e+09,9.864243e+09,1.610742e+09,3.378312e+09,7.037524e+08,4.520512e+09,1.111527e+08,1.515637e+09,1.940678e+09,4.046213e+09,...,6.638629e+08,4.360652e+09,7.074868e+08,3.932122e+09,3.399690e+08,3.903531e+09,6.349127e+09,6.346410e+09,83.093333,83.093333
1475444100,2.010894e+09,6.598088e+09,7.918703e+08,3.907536e+09,5.069260e+08,1.242612e+09,2.882337e+08,7.412996e+08,4.939713e+08,4.874295e+09,...,4.399758e+08,4.365551e+09,3.831572e+08,4.304758e+09,9.955344e+08,4.515371e+09,5.406501e+09,5.390604e+09,105.440000,105.440000
1475443800,1.094008e+09,3.205475e+09,4.054660e+08,5.830588e+08,3.189803e+08,5.642415e+08,2.680976e+08,9.504515e+08,1.113204e+09,4.697714e+09,...,3.019096e+08,3.985443e+09,6.328628e+08,3.828107e+09,5.150348e+08,4.815176e+09,4.095852e+09,4.067416e+09,96.533333,96.533333
1475443500,6.958911e+08,1.429079e+09,5.081743e+08,7.478002e+08,4.405091e+08,1.066645e+09,4.078161e+08,2.924433e+08,7.446744e+08,1.601454e+09,...,1.105860e+08,4.220654e+09,3.188105e+08,4.858653e+09,2.227317e+08,4.117597e+09,4.622143e+09,4.590731e+09,83.653333,83.653333
1475443200,1.099046e+09,1.284081e+09,6.027905e+08,2.485290e+09,2.592114e+08,1.599537e+09,1.813589e+08,1.655837e+09,1.390774e+09,1.361317e+09,...,3.261049e+08,4.217173e+09,4.820718e+08,4.011703e+09,4.296537e+08,4.127026e+09,4.689132e+09,4.653566e+09,120.000000,120.000000
1475442900,7.565415e+08,5.928482e+08,1.128569e+09,3.230383e+08,2.822930e+08,8.208347e+08,2.944739e+08,1.553013e+09,4.848643e+08,1.580601e+09,...,1.156634e+08,4.232073e+09,1.512990e+08,4.717033e+09,1.264105e+08,4.325710e+09,4.862466e+09,4.825648e+09,86.346667,86.346667
